In [5]:
# Import libraries
import glob
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np
import datetime
from dateutil import parser

In [6]:
# # Get CSV files list from a folder
# path = 'Weather_Data'
# csv_files = glob.glob(path + "/*.csv")

# df_list = (pd.read_csv(file).iloc[1:] for file in csv_files)

# # Concatenate all DataFrames
# weather_data = pd.concat(df_list, ignore_index=True)

# taking only one csv for now
path = "./Weather_Data/WeatherData_2022-06-30.csv"
weather_data = pd.read_csv(path)

In [7]:
# importing AIS dataset here
df_ais = pd.read_csv("./MarineTraffic/AIS_2022_06_30.csv")
df_ais.rename(columns={'LAT': 'latitude', 'LON': 'longitude'}, inplace=True)
df_ais.head()

,MMSI,BaseDateTime,latitude,longitude,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TransceiverClass
0,366999658,2022-06-30T00:00:00,30.12422,-85.63672,0.0,108.0,18,CG COBIA,NaN,NTXJ,90.0,0.0,26.0,5.0,NaN,55.0,A
1,538004028,2022-06-30T00:00:01,25.67896,-79.61538,9.2,180.2,182,RIO GRANDE,IMO9593438,V7UT9,80.0,0.0,274.0,48.0,17.1,80.0,A
2,367684960,2022-06-30T00:00:02,40.71821,-89.54574,0.0,360.0,511,CAROLE K,NaN,WDI3100,31.0,15.0,15.0,NaN,NaN,31.0,A
3,367533020,2022-06-30T00:00:03,31.10169,-91.60765,6.9,304.5,305,MARTHA LYNN,IMO8836637,WDG4104,31.0,12.0,54.0,15.0,NaN,57.0,A
4,368103510,2022-06-30T00:00:04,30.42336,-91.19565,0.0,239.3,511,ANDY MCKINNEY,NaN,WDK9416,31.0,0.0,24.0,10.0,3.0,31.0,A


In [8]:
weather_data.head()

,time,depth,latitude,longitude,Tdir,Tper,Thgt,sdir,sper,shgt,wdir,wper,whgt
0,UTC,m,degrees_north,degrees_east,degrees,second,meters,degrees,seconds,meters,degrees,seconds,meters
1,2022-06-30T00:00:00Z,0.0,-77.5,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-06-30T00:00:00Z,0.0,-77.5,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-06-30T00:00:00Z,0.0,-77.5,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-06-30T00:00:00Z,0.0,-77.5,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
def missing_statistics(df):
    statitics = pd.DataFrame(df.isnull().sum()).reset_index()
    statitics.columns = ['COLUMN NAME', "MISSING VALUES"]
    statitics['TOTAL ROWS'] = df.shape[0]
    statitics['% MISSING'] = round(
        (statitics['MISSING VALUES']/statitics['TOTAL ROWS'])*100, 2)
    return statitics


missing_statistics(weather_data)


,COLUMN NAME,MISSING VALUES,TOTAL ROWS,% MISSING
0,time,0,1119601,0.00
1,depth,0,1119601,0.00
2,latitude,0,1119601,0.00
3,longitude,0,1119601,0.00
4,Tdir,430536,1119601,38.45
5,Tper,430295,1119601,38.43
6,Thgt,430295,1119601,38.43
7,sdir,490930,1119601,43.85
8,sper,490930,1119601,43.85
9,shgt,490930,1119601,43.85


In [10]:
# Dropping rows with na
df_ais.dropna(inplace=True, axis=0)
weather_data.dropna(inplace=True, axis=0)
weather_data.reset_index(drop=True, inplace=True)

In [11]:
def convertISOtoLocal(isoString):
    return parser.parse(isoString).replace(tzinfo=datetime.timezone.utc).astimezone().strftime("%d/%m/%Y %I:%M:%S %p")


In [12]:
# converting all the ISO timestrings in
# "time" attribute to IST timezone
# weather_data["time"] = weather_data["time"].apply(lambda x: convertISOtoLocal(x))


In [13]:
weather_data.head()


,time,depth,latitude,longitude,Tdir,Tper,Thgt,sdir,sper,shgt,wdir,wper,whgt
0,UTC,m,degrees_north,degrees_east,degrees,second,meters,degrees,seconds,meters,degrees,seconds,meters
1,2022-06-30T00:00:00Z,0.0,-76.5,306.0,57.0,2.762431,0.29999998,123.0,2.9399998,0.049999997,56.0,2.69,0.29999998
2,2022-06-30T00:00:00Z,0.0,-76.0,213.5,180.0,1.7761989,0.08,200.0,1.92,0.06,133.0,1.43,0.06
3,2022-06-30T00:00:00Z,0.0,-76.0,303.5,59.0,2.770083,0.28,108.0,2.6599998,0.13,39.0,2.84,0.25
4,2022-06-30T00:00:00Z,0.0,-73.0,281.0,23.0,2.3364484,0.13,11.0,2.4299998,0.07,72.0,1.42,0.099999994


In [14]:
# getting unique values of time ranges
weather_data = weather_data.iloc[1:]
allTimeRanges = weather_data["time"].unique()

# removing all occurence of the word "UTC"
allTimeRanges = allTimeRanges[allTimeRanges != "UTC"]
allTimeRanges


array(['2022-06-30T00:00:00Z', '2022-06-30T01:00:00Z',
       '2022-06-30T02:00:00Z', '2022-06-30T03:00:00Z',
       '2022-06-30T04:00:00Z'], dtype=object)

In [15]:
# working on only 1 time range for now
x_subset = weather_data.loc[weather_data["time"] == allTimeRanges[0]]
x_subset.shape

(77461, 13)

In [16]:
# rounding all AIS lat and lng to nearest location 
def roundToNearestLoc(val):
    return round(val * 2) / 2

df_ais["latitude"] = df_ais["latitude"].apply(lambda x: roundToNearestLoc(x))
# adding 180 to the longitude to make it in the range of 0-360
df_ais["longitude"] = df_ais["longitude"].apply(lambda x: roundToNearestLoc(x) + 180)
df_ais[["latitude", "longitude"]]

,latitude,longitude
1,25.5,100.5
8,32.0,99.0
10,34.0,104.0
12,40.5,106.0
13,33.0,102.5
...,...,...
1048566,49.5,57.0
1048567,40.5,106.0
1048569,41.0,109.0
1048570,29.5,86.5


In [17]:
from pandas.api.types import is_numeric_dtype, is_string_dtype

weather_data["latitude"] = pd.to_numeric(weather_data["latitude"])
weather_data["longitude"] = pd.to_numeric(weather_data["longitude"])

In [18]:
# direct matches
df = pd.merge(weather_data, df_ais, on=['latitude','longitude'], how='outer', indicator='Exist')
df['Exist'] = np.where(df.Exist == 'both', True, False)


In [19]:
print("Exact match present:", round(((df["Exist"] == True).sum() / df.shape[0]) * 100), "%")
print("Exact match not present:", round(((df["Exist"] == False).sum() / df.shape[0]) * 100), "%")

Exact match present: 1 %
Exact match not present: 99 %


In [28]:
df.shape

(5848, 29)

In [29]:
df = df.loc[df["Exist"] == True]
# df.reset_index(inplace=True, drop= True)
df.head()
df.to_csv("Draft Cleaned Merged.csv", index=False)